In [ ]:
from mainnet_launch.database.schema.full import Destinations, AutopoolDestinations, Tokens, SwapQuote
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_highest_value_in_field_where,
)
from mainnet_launch.constants import *
import plotly.express as px
import streamlit as st
import plotly.io as pio
import pandas as pd

pio.templates.default = None


base_asset = WETH
chain = ETH_CHAIN

latest_quote_batch = get_highest_value_in_field_where(
    SwapQuote,
    SwapQuote.quote_batch,
    where_clause=(SwapQuote.base_asset == WETH(ETH_CHAIN)) & (SwapQuote.chain_id == ETH_CHAIN.chain_id),
)

swap_quotes_df = get_full_table_as_df(SwapQuote, where_clause=SwapQuote.quote_batch == latest_quote_batch)

In [ ]:
def display_swap_quotes_batch_meta_data(swap_quotes_df: pd.DataFrame) -> pd.DataFrame:
    """
    Display the swap quotes DataFrame with:
      • a one-line summary above (count, sources, batch, start → end, duration)
      • four metric cards (count, sources, batch, window)
      • the interactive table below
    """

    # compute metrics
    quote_count = len(swap_quotes_df)
    sources = swap_quotes_df["api_name"].unique().tolist()
    batch_number = swap_quotes_df["quote_batch"].iat[0] if quote_count > 0 else None
    start_time = swap_quotes_df["datetime_received"].min()
    end_time = swap_quotes_df["datetime_received"].max()
    window = end_time - start_time

    # format duration as Hh Mm Ss (or Mm Ss)
    total_seconds = int(window.total_seconds())
    hours, rem = divmod(total_seconds, 3600)
    minutes, sec = divmod(rem, 60)
    if hours:
        formatted_window = f"{hours}h {minutes}m {sec}s"
    else:
        formatted_window = f"{minutes}m {sec}s"

    # one-line summary
    st.markdown(
        f"**{quote_count} quotes** from **{', '.join(sources)}** in batch **{batch_number}**\n\n"
        f"📅 {start_time.strftime('%Y-%m-%d %H:%M:%S')} → {end_time.strftime('%Y-%m-%d %H:%M:%S')} "
        f"({formatted_window})"
    )

    # metrics cards
    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Quotes count", quote_count)
    c2.metric("Sources", ", ".join(sources))
    c3.metric("Batch #", batch_number)
    c4.metric("Duration", formatted_window)


def display_slippage_scatter(swap_quotes_df: pd.DataFrame):
    """
    Render a scatter plot of scaled_amount_in vs slippage_bps,
    filtered by sell_token_symbol and size_factor.
    """
    df = swap_quotes_df.copy()

    # Select sell token symbol
    symbol_options = df["sell_token_symbol"].unique().tolist()
    selected_symbol = st.selectbox("Select sell token symbol", symbol_options)

    # Select size factor (absolute / portion)
    factor_options = df["size_factor"].unique().tolist()
    selected_factor = st.selectbox("Select size factor", factor_options)

    # Filter the DataFrame
    filtered_df = df[(df["sell_token_symbol"] == selected_symbol) & (df["size_factor"] == selected_factor)]

    # Plot
    fig = px.scatter(
        filtered_df,
        x="scaled_amount_in",
        y="slippage_bps",
        color="label",
        title=f"Slippage vs Amount for {selected_symbol} ({selected_factor})",
    )
    st.plotly_chart(fig, use_container_width=True)

    return filtered_df


def _add_reference_price_column(swap_quotes_df: pd.DataFrame) -> None:

    # For each token, it is the min effective price for that swap size
    # the refernce price is the price of selling a non-trivial amount for the base asset
    # for stable coins it is 10_000 tokens, for ETH based assets it is 5 ETH
    # we get the median price for these swap sizes from the tokemak api at this batch

    smallest_amount_in = swap_quotes_df[swap_quotes_df["size_factor"] == "absolute"]["scaled_amount_in"].min()
    sell_token_to_reference_price = (
        swap_quotes_df[
            (swap_quotes_df["size_factor"] == "absolute")
            & (swap_quotes_df["scaled_amount_in"] == smallest_amount_in)
            & (swap_quotes_df["api_name"] == "tokemak")
        ]
        .groupby("sell_token_symbol")["effective_price"]
        .median()
        .to_dict()
    )
    swap_quotes_df["reference_price"] = swap_quotes_df["sell_token_symbol"].map(sell_token_to_reference_price)
    return sell_token_to_reference_price


def _add_token_symbols_columns(swap_quotes_df: pd.DataFrame) -> None:
    tokens_df = get_full_table_as_df(Tokens, where_clause=Tokens.chain_id == swap_quotes_df["chain_id"].iloc[0])
    token_address_to_symbol = dict(zip(tokens_df["token_address"], tokens_df["symbol"]))
    swap_quotes_df["buy_token_symbol"] = swap_quotes_df["buy_token_address"].map(token_address_to_symbol)
    swap_quotes_df["sell_token_symbol"] = swap_quotes_df["sell_token_address"].map(token_address_to_symbol)


def _augment_swap_quotes_df(swap_quotes_df: pd.DataFrame) -> pd.DataFrame:
    """
    Augment the swap quotes DataFrame with additional information.
    """
    # how much of the base asset we we get from selling X amoutn of the sell token
    # eg 101 sDAI / 100 USDC  -> effective price of 1.01 sDAI per USDC

    _add_token_symbols_columns(swap_quotes_df)
    _add_reference_price_column(swap_quotes_df)

    swap_quotes_df["effective_price"] = swap_quotes_df["scaled_amount_out"] / swap_quotes_df["scaled_amount_in"]
    swap_quotes_df["label"] = swap_quotes_df["sell_token_symbol"] + " - " + swap_quotes_df["api_name"]

    # - Sell a larger quantity (e.g., 100 stETH → receive 97.5 ETH)
    # - New price = 97.5 ETH ÷ 100 stETH = 0.975 ETH/stETH

    # - Excess slippage in basis points (bps):

    # `slippage_bps = 10 000 * (0.98 - 0.975) ÷ 0.98 ≈ 51 bps`
    swap_quotes_df["slippage_bps"] = (
        10_000
        * (swap_quotes_df["effective_price"] - swap_quotes_df["reference_price"])
        / swap_quotes_df["reference_price"]
    ).round(2)

    return swap_quotes_df


swap_quotes_df = _augment_swap_quotes_df(swap_quotes_df)

swap_quotes_df.head()

,chain_id,base_asset,api_name,sell_token_address,buy_token_address,scaled_amount_in,scaled_amount_out,pools_blacklist,aggregator_name,datetime_received,quote_batch,size_factor,id,effective_price,buy_token_symbol,sell_token_symbol,reference_price,label,slippage_bps
0,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,51.025267,53.804251,"('0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5',...",Odos,2025-08-08 16:08:06.814471+00:00,0,portion,5b6e56f7-bb6d-496d-ab74-05171e574cff,1.054463,WETH,ezETH,1.054590,ezETH - odos,-1.21
1,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,tokemak,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,12.756317,13.452206,None,Odos,2025-08-08 16:05:46.703884+00:00,0,portion,2e7fbb04-52ba-489e-a192-fd8b2dbb503b,1.054553,WETH,ezETH,1.054590,ezETH - tokemak,-0.36
2,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,99.707343,106.852611,"('0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5',...",Odos,2025-08-08 16:08:08.145069+00:00,0,portion,558ab109-9740-4672-9eca-b526f2e65624,1.071662,WETH,weETH,1.072348,weETH - odos,-6.40
3,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xbf5495Efe5DB9ce00f80364C8B423567e58d2110,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0.510253,0.538118,"('0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5',...",Odos,2025-08-08 16:05:36.254871+00:00,0,portion,d3220eda-5e2d-4893-881f-dc140ed09be2,1.054611,WETH,ezETH,1.054590,ezETH - odos,0.20
4,1,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,odos,0xae78736Cd615f374D3085123A210448E74Fc6393,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,42.420386,48.250531,"('0xdb74dfdd3bb46be8ce6c33dc9d82777bcfc3ded5',...",Odos,2025-08-08 16:08:08.504911+00:00,0,portion,801db96f-e6c7-48d1-8ef7-d65caeda6fbb,1.137437,WETH,rETH,1.137558,rETH - odos,-1.06


In [48]:
px.scatter(swap_quotes_df, x="scaled_amount_in", y="slippage_bps", color="label")

# the difference between the odos quote and the toekmak quote

(assuming odos is good at this pricing)

is because 

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_3rd_party.estimate_exit_liquidity_from_quotes import *
from mainnet_launch.constants import *

chain, base_asset, valid_autopools = (
    BASE_CHAIN,
    USDC,
    [a for a in ALL_AUTOPOOLS if a.chain == BASE_CHAIN and a.base_asset == USDC(BASE_CHAIN)],
)

unscaled_asset_exposure, percent_ownership_by_destination_df, token_df = fetch_needed_context(chain, valid_autopools)

(
    tokemak_quote_requests_df,
    odos_quote_requests_df,
    raw_tokemak_quote_response_df,
    raw_odos_quote_response_df,
    usdc_df,
) = fetch_odos_and_tokemak_quotes(chain=chain, base_asset=base_asset, valid_autopools=valid_autopools)
usdc_df2 = usdc_df.copy()
usdc_df

Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:04<00:00,  2.11it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:01<00:00,  6.16it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:04<00:00,  2.37it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


sleeping 2 minutes


Fetching 3rd-party data from https://swaps-pricing.tokemaklabs.com/swap-quote-v2: 100%|██████████| 10/10 [00:03<00:00,  2.55it/s]
Fetching 3rd-party data from https://api.odos.xyz/sor/quote/v2: 100%|██████████| 10/10 [00:05<00:00,  1.79it/s]

fetch_odos_and_tokemak_quotes took 265.9049 seconds.


,api,chain_id,token_in,token_out,token_in_symbol,token_out_symbol,scaled_amount_in,scaled_amount_out,datetime_received,pools_blacklist,aggregator_name,effective_price
0,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,200000.0,199423.465314,2025-08-07 19:43:04.411131+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.997117
1,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,20000.0,21016.695897,2025-08-07 19:43:04.415230+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.050835
2,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,20000.0,19982.141704,2025-08-07 19:43:04.419696+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.999107
3,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,200000.0,209267.785776,2025-08-07 19:43:04.422869+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.046339
4,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,100000.0,104926.216793,2025-08-07 19:43:04.524412+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.049262
5,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,50000.0,49945.885801,2025-08-07 19:43:04.564463+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.998918
6,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,10000.0,10510.005131,2025-08-07 19:43:04.568207+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.051001
7,odos,8453,0x646A737B9B6024e49f5908762B3fF73e65B5160c,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,scrvUSD,USDC,50000.0,52515.035467,2025-08-07 19:43:04.679508+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,1.050301
8,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,10000.0,9992.430118,2025-08-07 19:43:05.112601+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.999243
9,odos,8453,0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,GHO,USDC,100000.0,99866.206362,2025-08-07 19:43:05.210060+00:00,"(0x7ab124ec4029316c2a42f713828ddf2a192b36db, 0...",Odos,0.998662


In [29]:
usdc_df["label"] = usdc_df["token_in_symbol"] + " - " + df["api"]

agg_df = (
    usdc_df.groupby(["label", "token_in_symbol", "scaled_amount_in"])
    .agg(effective_price=("effective_price", "median"))
    .reset_index()
)


agg_df["reference_price"] = agg_df["token_in_symbol"].map(reference_price)
agg_df["slippage_bps_from_reference"] = (
    (agg_df["reference_price"] - agg_df["effective_price"]) / agg_df["reference_price"] * 10_000
)

agg_df.reset_index()

,index,label,token_in_symbol,scaled_amount_in,effective_price,reference_price,slippage_bps_from_reference
0,0,GHO - odos,GHO,10000.0,0.998323,NaN,NaN
1,1,GHO - odos,GHO,20000.0,0.998248,NaN,NaN
2,2,GHO - odos,GHO,50000.0,0.998153,NaN,NaN
3,3,GHO - odos,GHO,100000.0,0.998025,NaN,NaN
4,4,GHO - odos,GHO,200000.0,0.997246,NaN,NaN
5,5,scrvUSD - odos,scrvUSD,10000.0,1.049850,NaN,NaN
6,6,scrvUSD - odos,scrvUSD,20000.0,1.049684,NaN,NaN
7,7,scrvUSD - odos,scrvUSD,50000.0,1.049151,NaN,NaN
8,8,scrvUSD - odos,scrvUSD,100000.0,1.048116,NaN,NaN
9,9,scrvUSD - odos,scrvUSD,200000.0,1.045202,NaN,NaN


In [30]:
px.scatter(agg_df, x="scaled_amount_in", y="slippage_bps_from_reference", color="label")